In [1]:
import os 
import sys 
import zipfile

from skimage.io import imread

import numpy as np
import pandas as pd
import tensorflow as tf

import numpy as np
def scale_image(image1):
    im = image1.astype(np.float32)
    im /= 255
    return im

print(os.getcwd())

C:\Users\e7zv5qo\Jupyter_Notebooks\UCB_Deep_Learning_03\Module3


In [2]:
train_labels = pd.read_csv("./data/train_labels.csv")
print(train_labels)
train_images = list(train_labels['id'])
train_images = [ name + '.tif' for name in train_images]
labels = list(train_labels['label'])
print(train_images[0:5])

                                              id  label
0       f38a6374c348f90b587e046aac6079959adf3835      0
1       c18f2d887b7ae4f6742ee445113fa1aef383ed77      1
2       755db6279dae599ebb4d39a9123cce439965282d      0
3       bc3f0c64fb968ff4a8bd33af6971ecae77c75e08      0
4       068aba587a4950175d04c680d38943fd488d6a9d      0
...                                          ...    ...
220020  53e9aa9d46e720bf3c6a7528d1fca3ba6e2e49f6      0
220021  d4b854fe38b07fe2831ad73892b3cec877689576      1
220022  3d046cead1a2a5cbe00b2b4847cfb7ba7cf5fe75      0
220023  f129691c13433f66e1e0671ff1fe80944816f5a2      0
220024  a81f84895ddcd522302ddf34be02eb1b3e5af1cb      1

[220025 rows x 2 columns]
['f38a6374c348f90b587e046aac6079959adf3835.tif', 'c18f2d887b7ae4f6742ee445113fa1aef383ed77.tif', '755db6279dae599ebb4d39a9123cce439965282d.tif', 'bc3f0c64fb968ff4a8bd33af6971ecae77c75e08.tif', '068aba587a4950175d04c680d38943fd488d6a9d.tif']


In [3]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train_images, labels, random_state=0, train_size = 150000)
print(len(X_train))
print(len(y_train))
print(len(X_valid))
print(len(y_valid))
print(y_train[0:10])
print(np.mean(labels))

150000
150000
70025
70025
[1, 1, 0, 0, 0, 0, 0, 1, 0, 0]
0.40503124644926713


In [4]:
def read_image_collection(file_list, file_path):
    return np.array([
        scale_image(imread(file_path+file_name))
        for file_name in file_list])

In [5]:
print(X_train[0:5])

['2b21e7a0fae2f857c5a34f185b371e0db653fc44.tif', 'a050c93c3af5af6150e00e99b5d06413af6fbfd9.tif', '652c1edb16752b3c21085c87fda08989fde5f76e.tif', '64b8633a2bcfdacba25ffc30415130b20b779950.tif', '17987540920ea1427c9b3f0beeba2c1507d02632.tif']


In [24]:
X_train_images = read_image_collection(X_train, './data/train/')
print(X_train_images.shape)
X_valid_images = read_image_collection(X_valid, './data/train/')
print(X_valid_images.shape)
print('Ready')

(150000, 96, 96, 3)
(70025, 96, 96, 3)
Ready


In [7]:
gbytes = 150000 * 96 * 96 * 3 * 4/ 10**9
print(gbytes)

16.5888


In [32]:
from tensorflow import keras

def modelArch1():
    model = keras.models.Sequential([
        keras.layers.Conv2D(64, 3, activation='relu', padding='valid',
                           input_shape=[96,96,3]),
        keras.layers.MaxPool2D(2),
        keras.layers.Conv2D(128, 3, activation=tf.nn.relu, padding='valid'),
        keras.layers.Conv2D(128, 3,activation=tf.nn.relu, padding='valid'),
        keras.layers.MaxPool2D(2),
        keras.layers.Conv2D(256, 3, activation=tf.nn.relu, padding='valid'),
        keras.layers.Conv2D(256, 3, activation=tf.nn.relu, padding='valid'),
        keras.layers.MaxPool2D(2),
        keras.layers.Flatten(),
        keras.layers.Dense(128, activation=tf.nn.relu),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(64,  activation=tf.nn.relu),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(1, activation=tf.nn.sigmoid),
    ])
    return model

model = modelArch1()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 94, 94, 64)        1792      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 47, 47, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 45, 45, 128)       73856     
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 43, 43, 128)       147584    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 21, 21, 128)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 19, 19, 256)       295168    
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 17, 17, 256)      

In [33]:
opt = keras.optimizers.Adam(learning_rate=0.0005)
loss = keras.losses.BinaryCrossentropy()
model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

In [34]:
history = model.fit(x=X_train_images,y=np.array(y_train),verbose=1,validation_data=(X_valid_images,np.array(y_valid)),epochs=1)

4688/4688 [==============================] - 3277s 699ms/step - loss: 0.4618 - accuracy: 0.7862 - val_loss: 0.3847 - val_accuracy: 0.8340


In [40]:
history = model.fit(x=X_train_images,y=np.array(y_train),verbose=1,validation_data=(X_valid_images,np.array(y_valid)),epochs=1)

4688/4688 [==============================] - 3243s 692ms/step - loss: 0.3365 - accuracy: 0.8614 - val_loss: 0.2799 - val_accuracy: 0.8855


In [12]:
history = model.fit(x=X_train_images,y=np.array(y_train),verbose=1,validation_data=(X_valid_images,np.array(y_valid)),epochs=1)

In [14]:
sample = pd.read_csv('./data/sample_submission.csv')
sample_names = list(sample['id'])
sample_names[0:20]
## train_images = [ name + '.tif' for name in train_images]
test_images = [name + '.tif' for name in sample_names]
test_images[0:10]

['0b2ea2a822ad23fdb1b5dd26653da899fbd2c0d5.tif',
 '95596b92e5066c5c52466c90b69ff089b39f2737.tif',
 '248e6738860e2ebcf6258cdc1f32f299e0c76914.tif',
 '2c35657e312966e9294eac6841726ff3a748febf.tif',
 '145782eb7caa1c516acbe2eda34d9a3f31c41fd6.tif',
 '725dabe6ecccc68b958a2c7dd75bcbf362c7cb03.tif',
 'aa0307865281d4484ddf8c637c348292968b93a7.tif',
 'f4e5dc9c949920f1b3362982e15e99bf6f3ef83b.tif',
 '95e08c9cedc28a9b4a86f4fc1e06c1972134be08.tif',
 'ee1f5345a8d4e28403b7e61b97b5f76c201ce9cb.tif']

In [15]:
X_test_images = read_image_collection(test_images, './data/test/')
print(X_test_images.shape)

(57458, 96, 96, 3)


In [26]:
print(X_test_images.shape)

(57458, 96, 96, 3)


In [41]:
## predictions = (model.predict(X_test_images) > 0.5).astype("int32")
predict_proba = model.predict(X_test_images)
## predictions[0:10]
print(predict_proba[0:10])


[[0.04834113]
 [0.11054495]
 [0.0078705 ]
 [0.10122654]
 [0.2661528 ]
 [0.23163694]
 [0.3882869 ]
 [0.56499034]
 [0.02379045]
 [0.6751522 ]]


In [36]:
predictions = (predict_proba > 0.5).astype("int32")
print(predictions[0:10])
print(sum(predictions))
print(len(predictions))

[[0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]]
[25660]
57458


In [42]:
predictions = (predict_proba > 0.5).astype("int32")
print(predictions[0:10])
print(sum(predictions))
print(len(predictions))

[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]]
[18879]
57458


In [43]:
df = pd.DataFrame(sample_names)


In [44]:
df.columns= ['id']
df['label'] = predictions
df


,id,label
0,0b2ea2a822ad23fdb1b5dd26653da899fbd2c0d5,0
1,95596b92e5066c5c52466c90b69ff089b39f2737,0
2,248e6738860e2ebcf6258cdc1f32f299e0c76914,0
3,2c35657e312966e9294eac6841726ff3a748febf,0
4,145782eb7caa1c516acbe2eda34d9a3f31c41fd6,0
...,...,...
57453,061847314ded6f81e1cd670748bfa2003442c9c7,0
57454,6f3977130212641fd5808210015a609c658dcbff,0
57455,46935f247278539eca74b54d07d666efb528a753,0
57456,a09bcae08a82120183352e0e869181b2911d3dc1,0


In [45]:
df.to_csv('./data/submission4.csv',index=False)
